### FUNCTIONS / LIBRARIES

In [1]:
%load_ext autoreload
%autoreload 2


In [4]:
# Functions Optmization

from Optimization import prepare_vehicles_with_stats
from Optimization import spatial_optimization_pipeline
from Optimization import temporal_optimization_pipeline
from Optimization import run_fairness_pipeline
from Optimization import run_max_coverage_pipeline
from Optimization import combine_optimized_dfs
from Optimization import compute_combined_optimization_scores
from Optimization import select_random_vehicles
from Optimization import vehicle_optimization_stats_pipeline
from Optimization import extract_string_lists # Done (call if needed)
from Optimization import plot_vehicles_by_group # Done (call if needed)
from Optimization import prepare_selected_vehicles_from_combined 
from Optimization import master_function_analysis
from Optimization import visualization_master_function
from Optimization import pipeline_plot_frequency

import warnings
import geopandas as gpd
import pandas as pd

In [5]:
# Set the warning filter to ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)


### RAW DATA / PARAMETERS 

In [6]:
# Full CBS
cbs_full = gpd.read_file('data/temp/full_cbs.gpkg')

# CIty Stats

city_stats = pd.read_csv('data/city_stats_amsterdam.csv')

# Gruped by points 
points_gdf = gpd.read_file('data/temp/grouped_by_points_GVB.gpkg')

# Parameters
City = 'Amsterdam'

# City Boundary 
city_geo = gpd.read_file("data/Gemeente2.geojson")


### HERE YOU SET N for number of vehicles

In [7]:
# Set N
N = 100 # 50 and 100 

### PROCESS

In [8]:
# Usage Create Vehicles with Stats
vehicles_stats = prepare_vehicles_with_stats(points_gdf, cbs_full)

In [9]:
# Export Vehicle Stats
# vehicles_stats.to_file('data/temp/vehicles_stats_AMS_50buffer_1503.gpkg', driver='GPKG')

In [10]:
# Selecte Vehicles for Spatial Optimization # coverage threshold = 0 so that it continues!
optimized_spatial, filtered_vehicles, max_space_vehicles = spatial_optimization_pipeline(points_gdf, cbs_full, vehicles_stats, coverage_threshold=0, top_n=N)

In [11]:
# Select Vehicles for temporal optimization
optimized_temporal, filtered_vehicles_temp, max_temp_vehicles = temporal_optimization_pipeline(vehicles_stats, top_n=N)

In [12]:
# Selecte Vehicles for Fairness Optimization
closest_simple, closest_relative, closest_absolute, df_area_stats, df_opts, fair_vehicles = run_fairness_pipeline(vehicles_stats, city_stats, n=N)

c:\Users\pkoljensic\OneDrive - Delft University of Technology\Desktop\PYTHON\projects\sensing\sensing_env\Lib\site-packages\geopandas\geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\pkoljensic\OneDrive - Delft University of Technology\Desktop\PYTHON\projects\sensing\sensing_env\Lib\site-packages\geopandas\geodataframe.py:1819: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\pkoljensic\OneDrive - Delft University of Technology\Desk

In [13]:
# Selecte Vehicles for Max Coverage Optimization

tops, cbs_lists, max_number, gdf_filtered, max_pop_vehicles = run_max_coverage_pipeline(vehicles_stats, cbs_full, n=N)

max_A_inhab       = tops['max_A_inhab']
max_A_old         = tops['max_A_65+']
max_A_young       = tops['max_A_0_15']
max_A_dutch       = tops['max_A_nederlan']
max_A_non_western = tops['max_A_n_west_m']
max_P_old         = tops['max_P_65+']
max_P_non_western = tops['max_P_n_west_m']
max_point_count   = tops['max_count']

In [14]:
max_space_vehicles

,max_spatial
0,1921_GVB
1,309_GVB
2,2268_GVB
3,2021_GVB
4,1467_GVB
...,...
95,3040_GVB
96,3032_GVB
97,2115_GVB
98,1470_GVB


In [15]:
# Create Combined First Itearation 
combined_df = combine_optimized_dfs(max_space_vehicles, max_temp_vehicles, max_pop_vehicles, fair_vehicles)

In [16]:
combined_df

,max_spatial,max_temporal,max_A_inhab,max_A_65+,max_A_0_15,max_A_nederlan,max_A_n_west_m,max_P_65+,max_P_n_west_m,max_count,closest_absolute,closest_relative,closest_simple
0,1921_GVB,1416_GVB,1921_GVB,1921_GVB,2277_GVB,2268_GVB,2277_GVB,1423_GVB,1466_GVB,1416_GVB,1462_GVB,1462_GVB,1462_GVB
1,309_GVB,1607_GVB,2268_GVB,2268_GVB,2268_GVB,1921_GVB,1921_GVB,322_GVB,1104_GVB,2121_GVB,1917_GVB,1917_GVB,1917_GVB
2,2268_GVB,330_GVB,1906_GVB,1403_GVB,2257_GVB,1906_GVB,2257_GVB,1608_GVB,1118_GVB,2041_GVB,2019_GVB,2028_GVB,2089_GVB
3,2021_GVB,1606_GVB,2277_GVB,320_GVB,1921_GVB,1910_GVB,2265_GVB,1609_GVB,1138_GVB,1141_GVB,2089_GVB,2057_GVB,2057_GVB
4,1467_GVB,1413_GVB,1907_GVB,2277_GVB,2265_GVB,2277_GVB,1906_GVB,1604_GVB,367_GVB,315_GVB,2057_GVB,1419_GVB,2020_GVB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3040_GVB,1455_GVB,2019_GVB,3044_GVB,1957_GVB,2020_GVB,1124_GVB,1461_GVB,2103_GVB,1408_GVB,1168_GVB,1454_GVB,3065_GVB
96,3032_GVB,1125_GVB,2060_GVB,2121_GVB,3048_GVB,2019_GVB,1413_GVB,2064_GVB,2072_GVB,3055_GVB,1132_GVB,2098_GVB,3045_GVB
97,2115_GVB,1141_GVB,2057_GVB,2039_GVB,3044_GVB,2060_GVB,1953_GVB,1453_GVB,2124_GVB,1461_GVB,2147_GVB,1913_GVB,3044_GVB
98,1470_GVB,1117_GVB,1414_GVB,2049_GVB,3037_GVB,2057_GVB,1958_GVB,2037_GVB,2114_GVB,1907_GVB,3037_GVB,2037_GVB,3048_GVB


In [17]:
# Combined vehicles (based on chosen columns) - not included | May go for discussion in the paper
scores_combined_df, top_combined_final = compute_combined_optimization_scores(combined_df)

In [18]:
# Create Random Vehicles 
random_vehicles = select_random_vehicles(vehicles_stats, n=N) 

In [19]:
print(combined_df.columns.tolist())


['max_spatial', 'max_temporal', 'max_A_inhab', 'max_A_65+', 'max_A_0_15', 'max_A_nederlan', 'max_A_n_west_m', 'max_P_65+', 'max_P_n_west_m', 'max_count', 'closest_absolute', 'closest_relative', 'closest_simple']


In [20]:
# 1) build all_vehicles from gdf_vehicles, not vehicles_stats
all_vehicles = pd.DataFrame({
    'all_vehicles': vehicles_stats['uni_id'].astype(str)
})

# 2) reset its index to 0…177
all_vehicles = all_vehicles.reset_index(drop=True)

In [21]:
# Take random 10 vehicles from the combined DataFrame
#all_vehicles = all_vehicles.sample(n=50, random_state=1)


In [22]:
# Create Selected Vehicles Stats
final_df_cells_test = vehicle_optimization_stats_pipeline(
    vehicles_stats,
    cbs_full,
    city_stats,
    max_space_vehicles,
    max_temp_vehicles,
    max_pop_vehicles,
    fair_vehicles,
    top_combined_final, # as these are not included in the final analysis
    random_vehicles,
    all_vehicles
)

In [23]:

# Globally format all floats as integers with thousands separators
pd.set_option('display.float_format', '{:,.0f}'.format)

In [24]:
# We have tiny error that Amsterdam is not showing the right value 
final_df_cells_test.at['cells_unique', 'Amsterdam'] = len(cbs_full)
# We have tiny error that Amsterdam is not showing the right value 
final_df_cells_test.at['avg_points_per_cell', 'Amsterdam'] = '-'

In [25]:
# TWO VARIANTS for TEMPORAL - starting from max point vehicles - more points - or starting from max avg_points_per_cell - maybe better for the graph
final_df_cells_test

,max_spatial,max_temporal,max_A_inhab,max_A_65+,max_A_0_15,max_A_nederlan,max_A_n_west_m,max_P_65+,max_P_n_west_m,max_count,fairest_absolute,fairest_relative,fairest_simple,combined_opt,random,all_vehicles,Amsterdam
A_inhab,"3,369,775","4,635,185","5,045,025","5,021,630","4,930,190","5,039,050","4,780,770","2,308,995","3,320,540","2,986,315","3,397,900","3,400,640","3,385,360","560,085","3,069,285","8,471,845",-
A_0_15,"420,950","608,230","620,601","621,540","630,057","618,333","614,422","298,811","459,940","350,595","400,668","417,038","395,739","65,810","385,638","1,050,147",-
A_65+,"419,060","573,086","607,038","609,915","597,364","606,981","581,577","342,503","386,646","379,313","416,346","421,130","411,584","71,683","380,293","1,047,752",-
A_nederlan,"1,451,698","1,912,586","2,176,984","2,167,270","2,094,060","2,181,063","1,952,566","1,030,095","1,240,193","1,349,936","1,496,207","1,477,286","1,516,747","250,612","1,347,408","3,656,741",-
A_n_west_m,"1,207,541","1,810,292","1,800,426","1,793,378","1,819,808","1,786,645","1,870,848","792,077","1,473,611","975,172","1,156,796","1,201,149","1,117,794","186,587","1,065,478","3,006,099",-
count,"790,896","757,990","808,717","801,812","753,076","815,152","744,584","763,655","617,002","1,099,286","815,111","815,873","818,951","101,663","796,550","2,127,870",-
A_inhab_uniq,"403,240","328,640","382,665","385,145","382,340","385,310","381,360","274,380","291,420","363,965","371,570","374,800","360,410","284,865","388,385","403,240","870,375"
A_0_15_uniq,"52,298","43,007","48,976","49,288","49,575","49,334","49,439","35,862","40,060","46,655","47,787","48,601","46,522","36,270","50,210","52,298",122388
A_65+_uniq,"51,191","42,147","48,722","49,317","48,355","49,317","48,164","38,353","36,081","45,956","46,700","46,991","44,834","35,958","49,222","51,191",112084
A_nederlan_uniq,"178,541","143,714","169,905","171,374","167,504","171,452","167,004","123,237","120,087","159,417","163,218","165,473","157,539","124,810","171,231","178,541",382106


### WE DO IT MANUALLY HERE AND HERE COMES THE EXPORT

In [26]:
n = N
# Select and rename only the desired columns
subset_df = final_df_cells_test[[
    "max_spatial", 
    "max_temporal", 
    "fairest_relative"
]].copy()

subset_df.columns = [
    f"max_spatial_{n}",
    f"max_temporal_{n}",
    f"fairest_relative_{n}"
]


In [27]:
subset_df.index.name = "indicator"
subset_df.to_csv(f'data/temp/optimzed_vehicles_N{N}_test.csv', index=True, index_label=False)

In [28]:
subset_df

,max_spatial_100,max_temporal_100,fairest_relative_100
indicator,,,
A_inhab,"3,369,775","4,635,185","3,400,640"
A_0_15,"420,950","608,230","417,038"
A_65+,"419,060","573,086","421,130"
A_nederlan,"1,451,698","1,912,586","1,477,286"
A_n_west_m,"1,207,541","1,810,292","1,201,149"
count,"790,896","757,990","815,873"
A_inhab_uniq,"403,240","328,640","374,800"
A_0_15_uniq,"52,298","43,007","48,601"
A_65+_uniq,"51,191","42,147","46,991"


In [29]:
pd.read_csv(f'data/temp/optimzed_vehicles_N{N}_test.csv', index_col=0)

,max_spatial_100,max_temporal_100,fairest_relative_100
A_inhab,"3,369,775","4,635,185","3,400,640"
A_0_15,"420,950","608,230","417,038"
A_65+,"419,060","573,086","421,130"
A_nederlan,"1,451,698","1,912,586","1,477,286"
A_n_west_m,"1,207,541","1,810,292","1,201,149"
count,"790,896","757,990","815,873"
A_inhab_uniq,"403,240","328,640","374,800"
A_0_15_uniq,"52,298","43,007","48,601"
A_65+_uniq,"51,191","42,147","46,991"
A_nederlan_uniq,"178,541","143,714","165,473"


### EXPORT ALL VEHICLES AND CITY

In [36]:
# Extract only 'all_vehicles' and 'Amsterdam' columns
subset_df2 = final_df_cells_test[["all_vehicles", "Amsterdam"]].copy()
subset_df2.index.name = "indicator"
subset_df2.to_csv(f'data/temp/all_vehicles_Amsterdam_test.csv', index=True, index_label=False)

In [37]:
pd.read_csv(f'data/temp/all_vehicles_Amsterdam_test.csv', index_col=0)

,all_vehicles,Amsterdam
A_inhab,"8,471,845",-
A_0_15,"1,050,147",-
A_65+,"1,047,752",-
A_nederlan,"3,656,741",-
A_n_west_m,"3,006,099",-
count,"2,127,870",-
A_inhab_uniq,"403,240",870375.0
A_0_15_uniq,"52,298",122388
A_65+_uniq,"51,191",112084
A_nederlan_uniq,"178,541",382106


In [30]:
# THEN GO SEE THE TASKS TO DO FRO EXPORRTS
# GITHUB REPO Update + Share 
# TITUS | 

In [31]:
# Createa CSV with the selected vehicles
# combined_df.to_csv('data/temp/combined_df_10_AMS_1503.csv', index=False)

In [32]:
# # Usage Visualization Quick (Only if needed!)

# lists_dict = extract_string_lists(combined_df)
# lists_dict.keys()
# plot_vehicles_by_group(vehicles_stats, lists_dict, city_geo) 

In [31]:
# # Create Optmized Vehicles GDFs 
# max_space_gdf = prepare_selected_vehicles_from_combined(vehicles_stats, combined_df, column='max_temporal')
# # Create Main Analysis 
# sums_df, cbs_gdf, joined_gdf, average_stats = master_function_analysis(
#     max_space_gdf, cbs_full,50)
# # Create Main Visualization 
# fig1, fig2, fig3, fig4 = visualization_master_function(
#     max_space_gdf, cbs_full, joined_gdf, city_geo, 50, sums_df, average_stats)